## 目標
請撰寫一個程式來上網抓取ptt 八卦版的資料
網址: https://www.ptt.cc/bbs/Gossiping/index.html
撰寫程式語言不拘(建議用python)
對於網路爬蟲可以使用 jsoup 或是 beautiful soup 
要爬取的資料有 :
文章的作者ID、標題、內文、留言
共爬取40篇文章
將資料全部包成一個json格式的檔案

問題:
1. 特殊格式的文章
 - [公告] 水桶 之類的
2. 果然還是要先分析 目標 
 - 找到 要抓取的內容
 - 選擇分析方式

參考資料:
1. BeautifulSoup
 - https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- ptt-web-crawler (PTT 網路版爬蟲) 
 - https://github.com/jwlin/ptt-web-crawler\
 - https://www.ptt.cc/bbs/Python/M.1484321622.A.951.html
- 瞭解JSON格式
 - http://j796160836.pixnet.net/blog/post/30530326-%E7%9E%AD%E8%A7%A3json%E6%A0%BC%E5%BC%8F
- JSON UTF-8 格式儲存
 - https://stackoverflow.com/questions/18337407/saving-utf-8-texts-in-json-dumps-as-utf8-not-as-u-escape-sequence
- JSON 格式轉換 網站
 - http://jsoneditoronline.org/

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
import re
from six import u
import codecs
#import time

###### 目標網頁

In [ ]:
webLoca = "https://www.ptt.cc"
targetWeb = '/bbs/Gossiping/index.html'
fileNameJSON = 'output.json'

###### 函數

In [ ]:
def FindContent(soup, article_id):
    main_content = soup.find(id="main-content")
    # 移除 '※ 發信站:' (starts with u'\u203b'), '◆ From:' (starts with u'\u25c6'), 空行及多餘空白
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
    expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])

    filtered = [_f for _f in filtered if _f]  # remove empty strings
    #filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
    filtered_TEMP = []  # remove last line containing the url of the article
    count = 0
    for x in filtered :
        count+=1
        if count <=8: #切頭，文章訊息
            continue
        if article_id not in x: #斷尾，去掉文章連結後的留言
            filtered_TEMP.append(x)
            continue
    #    if count >= 0:
        break
    #    count +=1
    content = ' '.join(filtered_TEMP)
    content = re.sub(r'(\s)+', ' ', content)
    #print ('content:', content)
    return content

In [ ]:
#
def GetSpecifiedData_oneArticle(articleDivBlock):
    try:
        #print(webLoca + articleDivBlock.select('a')[0]['href'])
        articleRES = rs.get(webLoca  + '/' + articleDivBlock.select('a')[0]['href'], verify=False)
        articleSoup = BeautifulSoup(articleRES.text, 'lxml')
        #作者與標題
        try: 
            author = articleSoup.select('.article-meta-value')[0].text #作者ID
            title = articleSoup.select('.article-meta-value')[2].text #標題
            #print(author, title)
        except IndexError as err: #發生於 [公告] 12月八卦板置底閒聊文，沒有作者與標題
            #print (err, '公告文')
            author = None
            #title = None
            title = "公告"
            pass
        #內文
        try: 
            #print (articleSoup.select('#main-container'))
            article_id = articleDivBlock.select('a')[0]['href'].split('/')[-1]
            content = FindContent(articleSoup, re.sub('\.html', '', article_id))
            #print(content)
        except IndexError as err: #
            #print (err, '沒內文?')
            content = None
            #pass
        #留言
        push_content_list = []
        try: 
            for pushMsg in articleSoup.select('.push'):
                #push_userid = pushMsg.select('span[class^=f3]')[0].text #留言者
                push_content = pushMsg.select('span[class^=f3]')[1].text.split(':',1)[1] #留言
                push_content_list.append(push_content)
                #print(push_userid, push_content)
                #break
        except IndexError as err: #發生於沒有推文 #這好像沒用
            #print (err, '沒有推文')
            #push_userid = None
            push_content_list = None
            #pass

        #print(author, title)
        #print(content, push_content_list)
    except IndexError as err: #本文已刪除，無連結可進
        #print (err, '本文已刪除')
        title = articleDivBlock.select('.title')[0].text
        author = articleDivBlock.select('.author')[0].text
        content = None
        push_content_list = None
    
    data = {"author":author,
            "title":title,
            "content":content,
            "push_content_list": push_content_list
            }
    #time.sleep(0.1)
    return json.dumps(data, sort_keys=True, ensure_ascii=False)


In [ ]:
def StoreJSON(data, mode):
    global fileNameJSON
    with codecs.open(fileNameJSON, mode, encoding='utf-8') as f:
        f.write(data)

######  已滿18歲

In [ ]:
payload = { 'from': targetWeb, 
           'yes':'yes'
          }

In [ ]:
rs = requests.session()

In [ ]:
res = rs.post("https://www.ptt.cc/ask/over18", verify = False, data = payload)

###### 進入文章列表

###### 撈取資料

In [ ]:
#dataList = []
countArticleNum = 0
targetArticleNum = 40
chatArticleTF = True #跳過公告用
firstTF = True #辨認頭一組內容

In [ ]:
#開頭
StoreJSON('{','w')

res = rs.get(webLoca + targetWeb, verify = False)
soup = BeautifulSoup(res.text, 'lxml')
while countArticleNum < targetArticleNum:
    #print(webLoca +soup.select('a[class^=btn]')[3]['href'])
    for articleDivBlock in reversed(soup.select('.r-ent')): #從底記上來，越底越新 ？
        if chatArticleTF: #略過公告
            chatArticleTF=False
            continue
        countArticleNum += 1
        if firstTF: #辨認頭一組內容
            StoreJSON('\n' + '"'+str(countArticleNum)+'":[','a')
            firstTF=False
        else:
            StoreJSON(',\n' + '"'+str(countArticleNum)+'":[','a')
        data = GetSpecifiedData_oneArticle(articleDivBlock)
        StoreJSON(data,'a')
        StoreJSON(']','a')
        if countArticleNum >= targetArticleNum:
            break
    #換下一頁
    res = rs.get(webLoca + soup.select('a[class^=btn]')[3]['href'], verify=False)
    soup = BeautifulSoup(res.text, 'lxml')
            
#結尾
StoreJSON('\n}','a')

###### 儲存輸出
輸出函數已寫在上面

###### 顯示

In [ ]:
def getJSON(filename, mode='r'):
    with codecs.open(filename, mode, encoding='utf-8') as f:
        return json.load(f)

In [ ]:
print (json.dumps(getJSON(fileNameJSON), indent=4, sort_keys=True,ensure_ascii=False))